In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.optim import SGD

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
datafolder = './data'
fmnist = datasets.FashionMNIST(root=datafolder, download=True, train=True)

x = fmnist.data
y = fmnist.targets

In [4]:
class FmnistDataSet(Dataset):
    def __init__(self, x, y) -> None:
        x = x.float()/255
        x = x.view(-1, 28*28)
        self.x, self.y = x, y
    
    def __getitem__(self, index):
        return self.x[index].to(device), self.y[index].to(device)
    
    def __len__(self):
        return len(self.x)

train       = FmnistDataSet(x, y)
trainLoader = DataLoader(train, batch_size=32, shuffle=True)

In [5]:
class Cmodel(nn.Module):
    def __init__(self) -> None:
        super(Cmodel, self).__init__()
        self.fc1   = nn.Linear(28*28, 1000)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Linear(1000, 10)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        return x

cmodel      = Cmodel().to(device)
criterion   = nn.CrossEntropyLoss()
optim       = SGD(cmodel.parameters(), lr=.001)

In [6]:
def train_batch(x, y, model, opt, loss):
    model.train()

    pred = model(x)

    batch_loss = loss(pred, y)

    batch_loss.backward()

    opt.step()

    opt.zero_grad()

    return batch_loss.item()

@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [7]:
losses, accuracies = [], []

for epoch in range(5):
    epoch_loss, epoch_acc = [], []
    for i, (x, y) in enumerate(trainLoader):
        batch_loss = train_batch(x, y, cmodel, optim, criterion)
        epoch_loss.append(batch_loss)
    losses.append(np.array(epoch_loss).mean())
    for i, (x, y) in enumerate(trainLoader):
        acc = accuracy(x, y, cmodel)
        epoch_acc.extend(acc)
    accuracies.append(np.array(epoch_acc).mean())
    print(losses[-1], end='; ')
    print(accuracies[-1])


1.7299814961751303; 0.6714
1.0845257649739584; 0.70035
0.8759701680501302; 0.7348833333333333
0.7806647875308991; 0.7552666666666666
0.7219484290917715; 0.7712333333333333
